In [2]:
# Dependencies

import pandas as pd
import matplotlib
import json
import requests
import time
import numpy as np
import pprint
from scipy.stats import linregress

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
SPOTIPY_CLIENT_ID = '804be4acb7d8409fb21a9ac6568a8c81'
SPOTIPY_CLIENT_SECRET = 'd2691d72912c464dae450bb998501b6d'

# Spotify data files
data_csv_path = "Resources/data.csv"
data_by_genres_csv_path = "Resources/data_by_genres.csv"

# Read the raw spotify csv
data = pd.read_csv(data_csv_path)
data_by_genres = pd.read_csv(data_by_genres_csv_path)

# Verify data completeness and headers
data.count()
data.head(50)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.2240,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
5,0.996000,['Mamie Smith & Her Jazz Hounds'],0.424,198627,0.2450,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920
6,0.992000,['Mamie Smith'],0.782,195200,0.0573,0,5DlCyqLyX2AOVDTjjkDZ8x,0.000002,5,0.1760,-12.453,1,Don't You Advertise Your Man,5,1920,0.0592,85.652,0.4870,1920
7,0.996000,['Mamie Smith & Her Jazz Hounds'],0.474,186173,0.2390,0,02FzJbHtqElixxCmrpSCUa,0.186000,9,0.1950,-9.712,1,Arkansas Blues,0,1920,0.0289,78.784,0.3660,1920
8,0.996000,['Francisco Canaro'],0.469,146840,0.2380,0,02i59gYdjlhBmbbWhf8YuK,0.960000,8,0.1490,-18.717,1,La Chacarera - Remasterizado,0,1920-07-08,0.0741,130.060,0.6210,1920
9,0.006820,['Meetya'],0.571,476304,0.7530,0,06NUxS2XL3efRh0bloxkHm,0.873000,8,0.0920,-6.943,1,Broken Puppet - Original Mix,0,1920-01-01,0.0446,126.993,0.1190,1920


In [3]:
# Verify data_by_genres completeness and headers
data_by_genres.count()
data_by_genres.head(50)

,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode
0,21st century classical,0.754600,0.284100,3.525932e+05,0.159580,0.484374,0.168580,-22.153400,0.062060,91.351000,0.143380,6.600000,4,1
1,432hz,0.485515,0.312000,1.047430e+06,0.391678,0.477250,0.265940,-18.131267,0.071717,118.900933,0.236483,41.200000,11,1
2,8-bit,0.028900,0.673000,1.334540e+05,0.950000,0.630000,0.069000,-7.899000,0.292000,192.816000,0.997000,0.000000,5,1
3,[],0.535793,0.546937,2.495312e+05,0.485430,0.278442,0.220970,-11.624754,0.101511,116.068980,0.486361,12.350770,7,1
4,a cappella,0.694276,0.516172,2.018391e+05,0.330533,0.036080,0.222983,-12.656547,0.083627,105.506031,0.454077,39.086248,7,1
5,abstract,0.411262,0.473600,3.660984e+05,0.479400,0.841400,0.097160,-15.799400,0.045300,142.495400,0.274260,43.800000,5,1
6,abstract beats,0.459000,0.669000,1.748670e+05,0.735000,0.000000,0.103000,-7.264000,0.329000,117.284000,0.723000,62.000000,10,1
7,abstract hip hop,0.297353,0.725274,2.282049e+05,0.726844,0.099187,0.268938,-7.257105,0.224437,107.588773,0.639051,21.050866,1,1
8,abstract idm,0.995000,0.174000,5.805330e+05,0.002910,0.931000,0.098300,-37.518000,0.039900,67.560000,0.032800,46.000000,0,1
9,accordeon,0.598750,0.442000,1.654800e+05,0.314000,0.607000,0.191675,-12.174250,0.042475,139.819500,0.511500,35.250000,2,1


In [4]:
# Clean data
clean_data = data.drop_duplicates(subset=['id','name'], keep=False)
clean_data.head(50)

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.2240,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.5170,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.1860,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.7980,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.7070,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
5,0.996000,['Mamie Smith & Her Jazz Hounds'],0.424,198627,0.2450,0,3HnrHGLE9u2MjHtdobfWl9,0.799000,5,0.2350,-11.470,1,Crazy Blues - 78rpm Version,9,1920,0.0397,103.870,0.4770,1920
6,0.992000,['Mamie Smith'],0.782,195200,0.0573,0,5DlCyqLyX2AOVDTjjkDZ8x,0.000002,5,0.1760,-12.453,1,Don't You Advertise Your Man,5,1920,0.0592,85.652,0.4870,1920
7,0.996000,['Mamie Smith & Her Jazz Hounds'],0.474,186173,0.2390,0,02FzJbHtqElixxCmrpSCUa,0.186000,9,0.1950,-9.712,1,Arkansas Blues,0,1920,0.0289,78.784,0.3660,1920
8,0.996000,['Francisco Canaro'],0.469,146840,0.2380,0,02i59gYdjlhBmbbWhf8YuK,0.960000,8,0.1490,-18.717,1,La Chacarera - Remasterizado,0,1920-07-08,0.0741,130.060,0.6210,1920
9,0.006820,['Meetya'],0.571,476304,0.7530,0,06NUxS2XL3efRh0bloxkHm,0.873000,8,0.0920,-6.943,1,Broken Puppet - Original Mix,0,1920-01-01,0.0446,126.993,0.1190,1920


In [5]:
# Filter by year and count of songs by year
filter_2019 = clean_data.loc[clean_data['year'] == 2019]
length_2019 = len(filter_2019)
print(length_2019)

filter_2020 = clean_data.loc[clean_data['year'] == 2020]
length_2020 = len(filter_2020)
print(length_2020)

filter_2021 = clean_data.loc[clean_data['year'] == 2021]
length_2021 = len(filter_2021)
print(length_2021)


2159
3124
988


In [6]:
summary_key_2019 = filter_2019.groupby('key')

# Calculate the mean, standard deviation by year for popularity of key
key_mean_2019 = round(summary_key_2019['popularity'].mean(),2)
key_stdev_2019 = round(summary_key_2019['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_key_mean_2019 = pd.DataFrame({'Mean': key_mean_2019,
                                   'Standard Deviation': key_stdev_2019})

summary_key_mean_2019.head(20)

,Mean,Standard Deviation
key,,
0,38.36,32.37
1,37.51,32.84
2,31.35,32.04
3,35.36,33.86
4,31.15,31.48
5,37.51,32.71
6,39.49,32.47
7,32.56,31.47
8,41.43,33.37


In [7]:
summary_key_2020 = filter_2020.groupby('key')

# Calculate the mean, standard deviation by year for popularity of key
key_mean_2020 = round(summary_key_2020['popularity'].mean(),2)
key_stdev_2020 = round(summary_key_2020['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_key_mean_2020 = pd.DataFrame({'Mean': key_mean_2020,
                                   'Standard Deviation': key_stdev_2020,})

summary_key_mean_2020.head(20)

,Mean,Standard Deviation
key,,
0,29.24,32.22
1,38.98,32.67
2,25.78,31.48
3,25.32,30.73
4,31.78,34.69
5,32.50,33.24
6,35.93,32.64
7,25.91,29.47
8,33.43,33.83


In [8]:
summary_key_2021 = filter_2021.groupby('key')

# Calculate the mean, standard deviation by year for popularity of key
key_mean_2021 = round(summary_key_2021['popularity'].mean(),2)
key_stdev_2021 = round(summary_key_2021['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_key_mean_2021 = pd.DataFrame({'Mean': key_mean_2021,
                                   'Standard Deviation': key_stdev_2021})

summary_key_mean_2021.head(20)

,Mean,Standard Deviation
key,,
0,9.49,22.59
1,10.08,23.12
2,16.60,27.93
3,11.65,23.97
4,12.77,25.52
5,9.05,20.44
6,15.88,27.10
7,13.46,25.32
8,14.73,24.61


In [9]:
# Legend of keys by number

key_scale = pd.DataFrame({"key": [0,1,2,3,4,5,6,7,8,9,10,11], 
                          "scale": ["C", "C#/Db", "D", "D#/Eb", "E", "F", 
                                    "F#/Gb", "G", "G#/Ab", "A", "A#/Bb", "B"]})
print(key_scale)

    key  scale
0     0      C
1     1  C#/Db
2     2      D
3     3  D#/Eb
4     4      E
5     5      F
6     6  F#/Gb
7     7      G
8     8  G#/Ab
9     9      A
10   10  A#/Bb
11   11      B


In [10]:
# Combine years into dataframe
merge_key_2019_2020 = pd.merge(summary_key_mean_2019, 
                               summary_key_mean_2020, 
                               how ="outer",on = ['key'])
print(merge_key_2019_2020)

merge_key_all = pd.merge(merge_key_2019_2020, 
                               summary_key_mean_2021, 
                               how ="outer",on = ['key'])
print(merge_key_all)

     Mean_x  Standard Deviation_x  Mean_y  Standard Deviation_y
key                                                            
0     38.36                 32.37   29.24                 32.22
1     37.51                 32.84   38.98                 32.67
2     31.35                 32.04   25.78                 31.48
3     35.36                 33.86   25.32                 30.73
4     31.15                 31.48   31.78                 34.69
5     37.51                 32.71   32.50                 33.24
6     39.49                 32.47   35.93                 32.64
7     32.56                 31.47   25.91                 29.47
8     41.43                 33.37   33.43                 33.83
9     26.18                 31.34   25.96                 30.68
10    37.74                 34.41   30.22                 31.91
11    31.01                 32.31   30.72                 32.98
     Mean_x  Standard Deviation_x  Mean_y  Standard Deviation_y   Mean  \
key                           

In [11]:
# Mode evaluation 2019

summary_mode_2019 = filter_2019.groupby('mode')

# Calculate the mean, standard deviation by year for popularity of mode
mode_mean_2019 = round(summary_mode_2019['popularity'].mean(),2)
mode_stdev_2019 = round(summary_mode_2019['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_mode_mean_2019 = pd.DataFrame({'Mean': mode_mean_2019,
                                   'Standard Deviation': mode_stdev_2019})

summary_mode_mean_2019.head(20)

,Mean,Standard Deviation
mode,,
0,33.07,32.88
1,35.56,32.38


In [12]:
# mode evaluation 2020

summary_mode_2020 = filter_2020.groupby('mode')

# Calculate the mean, standard deviation by year for popularity of mode
mode_mean_2020 = round(summary_mode_2020['popularity'].mean(),2)
mode_stdev_2020 = round(summary_mode_2020['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_mode_mean_2020 = pd.DataFrame({'Mean': mode_mean_2020,
                                   'Standard Deviation': mode_stdev_2020})

summary_mode_mean_2020.head(20)

,Mean,Standard Deviation
mode,,
0,32.01,32.79
1,29.93,32.20


In [13]:
# mode evaluation 2021

summary_mode_2021 = filter_2021.groupby('mode')

# Calculate the mean, standard deviation by year for popularity of mode
mode_mean_2021 = round(summary_mode_2021['popularity'].mean(),2)
mode_stdev_2021 = round(summary_mode_2021['popularity'].std(),2)

# Create DataFrame to summarize calculations
summary_mode_mean_2021 = pd.DataFrame({'Mean': mode_mean_2021,
                                   'Standard Deviation': mode_stdev_2021})

summary_mode_mean_2021.head(20)

,Mean,Standard Deviation
mode,,
0,6.89,18.96
1,15.56,26.70


In [14]:
# Combine years into dataframe
merge_mode_2019_2020 = pd.merge(summary_mode_mean_2019, 
                               summary_mode_mean_2020, 
                               how ="outer",on = ['mode'])
print(merge_mode_2019_2020)

merge_mode_all = pd.merge(merge_mode_2019_2020, 
                               summary_mode_mean_2021, 
                               how ="outer",on = ['mode'])
print(merge_mode_all)

      Mean_x  Standard Deviation_x  Mean_y  Standard Deviation_y
mode                                                            
0      33.07                 32.88   32.01                 32.79
1      35.56                 32.38   29.93                 32.20
      Mean_x  Standard Deviation_x  Mean_y  Standard Deviation_y   Mean  \
mode                                                                      
0      33.07                 32.88   32.01                 32.79   6.89   
1      35.56                 32.38   29.93                 32.20  15.56   

      Standard Deviation  
mode                      
0                  18.96  
1                  26.70  


In [18]:
bar_key_all = merge_key_all.groupby(["key"]).count()["popularity"]

plot_pandas = bar_key_all.plot.bar(figsize=(15,10), color='b',fontsize =14)
bar_key_all
plt.xlabel("popularity",fontsize = 14)
plt.ylabel("key",fontsize = 14)
plt.title("Key Popularity",fontsize = 20)

plt.savefig("../Images/key_by_year.png", bbox_inches = "tight")
plt.tight_layout()
plt.show()

bar_key_all

KeyError: 'popularity'

In [13]:
birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id =SPOTIPY_CLIENT_ID, client_secret = SPOTIPY_CLIENT_SECRET ))
results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
   results = spotify.next(results)
   albums.extend(results['items'])
for album in albums:
   print(album['name'])

Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [ ]:
# Download Spotify Data CSV
